In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import csv
import json

%matplotlib inline

In [2]:
def softmax(x, temp=1):
#     x should be as long as number of possible clicks, each possibility given a value (
#     the x.max() is there for stability
    ex = np.exp((x - x.max()) / temp)
    return ex / ex.sum()

In [3]:
cd ..\experiments\data\human\1.8

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\data\human\1.8


In [4]:
df=pd.read_csv('trials.csv', sep=',')

In [5]:
import ast
toList = lambda x: ast.literal_eval(x)
df['clicks'] = df['clicks'].apply(toList)

In [6]:
cd ..\..\..\exp1\static\json

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\exp1\static\json


In [7]:
with open('rewards_0.10.json') as f:
    rewards_low = json.load(f)

with open('rewards_1.25.json') as f:
    rewards_med = json.load(f)
    
with open('rewards_4.00.json') as f:
    rewards_high = json.load(f)

In [8]:
def get_rewards_for_trial(reward_type, trial_id):
    for trial in reward_type['train']:
        if trial['trial_id'] == trial_id:
            return trial['stateRewards']
    for trial in reward_type['test']:
        if trial['trial_id'] == trial_id:
            return trial['stateRewards']

In [9]:
rewards = []

for trial in rewards_low['train']:
    rewards += trial['stateRewards']
for trial in rewards_low['test']:
    rewards += trial['stateRewards']    
    
np.mean(rewards)

-0.52941176470588236

In [10]:
(-0.029411764705882353 + -0.47058823529411764 + -0.52941176470588236)/3

-0.34313725490196073

## Likelihood Model

In [11]:
#Note: Only continues depth-first based off of the last click 
#TODO: Check if half-explored arms are eventually explored to completion
def depth_first(state, cur_clicks, dist, strategy_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0] #works only because indices represent location
    possible_clicks = []
    if len(cur_clicks) == 0:
        possible_clicks += [1, 5, 9, 13]
    else:
        last_click = cur_clicks[-1]
        if last_click in [1, 5, 9, 13]:
            if last_click+1 in observed_states:
                for click in [last_click+2, last_click+3]:
                    if (click not in observed_states):
                        possible_clicks.append(click)
                if len(possible_clicks) == 0:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states):
                            possible_clicks.append(click)
            else:
                possible_clicks.append(last_click+1)            
        else:
            if last_click+1 in observed_states:
                if last_click+2 in observed_states:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states):
                            possible_clicks.append(click)
                elif (last_click+2 < 17):
                    possible_clicks.append(last_click+2)
                else:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states):
                            possible_clicks.append(click)
            elif (last_click+1 < 17):
                possible_clicks.append(last_click+1)
            else:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states):
                        possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
            elif dist == 'Softmax':
                prob_next_click[possibility] = 1
    return prob_next_click

In [12]:
test_state = np.array([0,1,5,0,20,0,0,0,0,0,0,0,0,1,0,-10,-10])
res = depth_first(test_state, [], 'Softmax', None)
res

array([ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.])

In [13]:
def breadth_first(state, cur_clicks, dist, strategy_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    if len(cur_clicks) == 0:
        possible_clicks += [1, 5, 9, 13]
    else:
        last_click = cur_clicks[-1]
        if last_click in [1, 5, 9, 13]:
            if set([1, 5, 9, 13]).issubset(set(observed_states)):
                if set([2, 6, 10, 14]).issubset(set(observed_states)):
                    for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                        if (click not in observed_states):
                            possible_clicks.append(click)
                else:
                    for click in [2, 6, 10, 14]:
                        if (click not in observed_states):
                                possible_clicks.append(click)
            else:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states):
                            possible_clicks.append(click)
        elif last_click in [2, 6, 10, 14]:
            if set([2, 6, 10, 14]).issubset(set(observed_states)):
                for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states):
                        possible_clicks.append(click)
            else:
                for click in [2, 6, 10, 14]:
                    if (click not in observed_states):
                        possible_clicks.append(click)
        else:
            for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states):
                        possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
            elif dist == 'Softmax':
                prob_next_click[possibility] = 1
    return prob_next_click

In [14]:
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = breadth_first(test_state, [], 'Uniform', None)
res

array([ 0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,
        0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ])

In [88]:
def best_first(state, cur_clicks, dist, strategy_t):
    prob_next_click = np.zeros(17)
    observed_states = cur_clicks
    possible_clicks = []
    expected_reward_value = -0.343 # expected reward calculated by taking the mean of each trial's rewards
    if len(cur_clicks) == 0:
        possible_clicks += [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    else:
        for i in np.arange(1,17):
            if (i not in observed_states):
                possible_clicks.append(i)
    sim_state = np.copy(state)
    for i in np.arange(1, 17):
        if state[i] == 0:
            sim_state[i] = expected_reward_value
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(sim_state[indices])
                indices[1] += 1
                path_2 = np.sum(sim_state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(sim_state[indices])
                indices[1] += 1
                path_2 = np.sum(sim_state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(sim_state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(sim_state[indices])
    return prob_next_click

In [89]:
#state = np.array([0, 0, 20, 0, 0, 12, -8, 0, 0, -4, 0, 0, 0, 10, 0, 0, 0])
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = best_first(test_state, [1, 4, 16, 2, 13], 'Uniform', None)
res

array([ 0.,  0.,  0.,  6.,  0.,  0.,  5.,  5.,  5.,  0., -3., -3., -3.,
        0., -9.,  1.,  0.])

In [90]:
def backwards_best(state, cur_clicks, dist, strategy_t):
    prob_next_click = np.zeros(17)
    observed_states = cur_clicks
    possible_clicks = []
    leaves = [3, 4, 7, 8, 11, 12, 15, 16]
    expected_reward_value = -0.343 # expected reward calculated by taking the mean of each trial's rewards
    goal = False
    goal_node = None
    if len(cur_clicks) == 0:
        possible_clicks += leaves
    else:
        for leaf in leaves: # should the leaves be examined in the order clicked?
            if state[leaf] > strategy_t: #a goal node has been identified
                #check that the whole arm hasn't been explored before marking goal as true
                if leaf in [3, 7, 11, 15]:
                    if (leaf-1 not in observed_states) or (leaf-2 not in observed_states):
                        goal = True
                        goal_node = leaf
                        break
                else:
                    if (leaf-2 not in observed_states) or (leaf-3 not in observed_states):
                        goal = True
                        goal_node = leaf
                        break
        if goal:
            #go backwards on that arm
            if goal_node in [3, 7, 11, 15]:
                if goal_node-1 not in observed_states:
                    possible_clicks.append(goal_node-1)
                else:
                    possible_clicks.append(goal_node-2)
            else:
                if goal_node-2 not in observed_states:
                    possible_clicks.append(goal_node-2)
                else:
                    possible_clicks.append(goal_node-3)
        else:
            for leaf in leaves:
                if leaf not in observed_states:
                    possible_clicks.append(leaf)
            
    sim_state = np.copy(state)
    for i in np.arange(1, 17):
        if state[i] == 0:
            sim_state[i] = expected_reward_value
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(sim_state[indices])
                indices[1] += 1
                path_2 = np.sum(sim_state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(sim_state[indices])
                indices[1] += 1
                path_2 = np.sum(sim_state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(sim_state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(sim_state[indices])
    return prob_next_click

In [74]:
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = backwards_best(test_state, [1, 4, 16, 2, 13], 'Uniform', 0)
res

array([ 0.,  0.,  0.,  6.,  0.,  0.,  0.,  5.,  5.,  0.,  0., -3., -3.,
        0.,  0.,  0.,  0.])

In [75]:
def prune(threshold, state):
    #returns nodes on pruned branches
    pruned = []
    observed_states = np.nonzero(state)[0]
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    direction = False
    if len(observed_states) != 0:
        for loc in observed_states:
            if (loc in [1, 2]) and (loc not in pruned):
                direction = bottom
            elif (loc in [5, 6]) and (loc not in pruned):
                direction = right
            elif (loc in [9, 10]) and (loc not in pruned):
                direction = top
            elif (loc in [13, 14]) and (loc not in pruned):
                direction = left
            if direction:
                path_val = np.sum(state[direction])
                if path_val < threshold:
                    pruned += direction
            else: #prune arm when both leaf nodes have a value below the threshold
                if (loc in [3, 7, 11, 15]) and (loc not in pruned):
                    if (state[loc] < threshold) and (state[loc+1] < threshold):
                        pruned += [loc-2, loc-1, loc, loc+1]
                elif (loc not in pruned):
                    if (state[loc] < threshold) and (state[loc-1] < threshold):
                        pruned += [loc-3, loc-2, loc-1, loc]
    return pruned        

In [76]:
def satisficing_stop(state, threshold):
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    for direction in [bottom, top, right, left]:
        observed_value = np.sum(state[direction])
        if observed_value > threshold:
            return True
    return False    

In [131]:
def strategy_likelihood(click_seq, strategy, cost_per_click, rewards, p_error, soft_temp=1,
                        log=False, dist='Uniform', strategy_t=-20, pruned=(0.5,-30), stop=(0,0)):
    #stop should be a tuple of (stopping rule type, threshold)
    #pruned is a tuple of (pruning probability, pruning threshold) where pruning probability > 0.5
    if log:
        prob = 0
    else:
        prob = 1
    cur_state = np.zeros(17)
    cur_clicks = []
    stop_type = stop[0]
    stop_threshold = stop[1]
    c_seq = list(click_seq) #don't want to modify the actual data
    c_seq.append(0) #0 serves as the termination action
    for click in c_seq:
        next_state = np.copy(cur_state)
        possible_next_clicks = strategy(cur_state, cur_clicks, dist, strategy_t)
        pruned_nodes = prune(pruned[1], cur_state)
        next_state[click] = rewards[click]
        n_possible_clicks = np.count_nonzero(possible_next_clicks) #this maybe the problem but?
        if (strategy.__name__ == 'best_first') or (strategy.__name__ == 'backwards_best'):
            n_possible_clicks -= len(cur_clicks)
        if stop_type == 1: # number of clicks x cost per click
            if (len(cur_clicks)*cost_per_click) > stop_threshold:
                possible_next_clicks = np.zeros(17)
        elif stop_type == 2: #satisficing
            stop_flag = satisficing_stop(cur_state, stop_threshold)
            if stop_flag:
                possible_next_clicks = np.zeros(17)
                possible_next_clicks[0] = 1
        if dist == 'Uniform':
            if (strategy.__name__ == 'best_first') or (strategy.__name__ == 'backwards_best'):
                possible_next_clicks = softmax(possible_next_clicks, soft_temp)
            for n in pruned_nodes:
                possible_next_clicks[n] *= (1-pruned[0])
                if np.sum(possible_next_clicks) != 0:
                    possible_next_clicks[n] /= np.sum(possible_next_clicks)
            p_strat = possible_next_clicks[click]
            if (click == 0) and ~np.all(possible_next_clicks):
                p_strat = 1
            num_wrong_clicks = 16 - (len(cur_clicks)+n_possible_clicks)
            p_click_given_state = (((1-p_error)*p_strat) + (p_error/(num_wrong_clicks+0.00000000001)))
        elif dist == 'Softmax':
            prob_next_clicks = softmax(possible_next_clicks, soft_temp)
            for n in pruned_nodes:
                prob_next_clicks[n] *= (1-pruned[0])
            if np.sum(possible_next_clicks) != 0:
                prob_next_clicks[n] /= np.sum(prob_next_clicks)
            p_click_given_state = prob_next_clicks[click]
        if log:
            prob += np.log(p_click_given_state)
        else:
            prob *= p_click_given_state
        cur_state = next_state
        cur_clicks.append(click)
    return prob

In [129]:
rewards = get_rewards_for_trial(rewards_med, df.values[1435][6])
strategy_likelihood(df.values[1435][10], best_first, 1.25, rewards, 0.1, 1, log=True, dist='Uniform')

17.124745808150674

In [132]:
data = df[df['info_cost'] == 1.25]
all_likelihoods = np.zeros((len(data.values), 6))

p_error = 0.1
softmax_temp = 1
satisfice = 0
p_prune = 0.5
prune_t = -30
for i in np.arange(len(data.values)):
    trial = data.values[i]
    click_cost = trial[2]
    all_likelihoods[i, 0] = trial[1] #pid
    all_likelihoods[i, 1] = click_cost
    click_seq = trial[10]
    trial_id = trial[6]
    if click_cost == 0.10:
        rewards = get_rewards_for_trial(rewards_low, trial_id)
    elif click_cost == 1.25:
        rewards = get_rewards_for_trial(rewards_med, trial_id)
    elif click_cost == 4.00:
        rewards = get_rewards_for_trial(rewards_high, trial_id)
    strategies = [depth_first, breadth_first, best_first, backwards_best]
    for j in np.arange(4):
        strat = strategies[j]
        likelihood = strategy_likelihood(click_seq, strat, click_cost, rewards, p_error, softmax_temp, log=True, dist='Uniform', stop=(2, satisfice), pruned=(p_prune, prune_t))
        all_likelihoods[i, j+2] = likelihood


In [133]:
likelihood_df = pd.DataFrame(all_likelihoods, columns=['pid', 'click_cost', 'depth_first', 
                                                       'breadth_first', 'best_first', 'backwards_best' ])

In [134]:
#np.mean(likelihood_df['breadth_first'])
likelihood_df

,pid,click_cost,depth_first,breadth_first,best_first,backwards_best
0,0.0,1.25,-1.552743,-1.551431,21.559208,-4.491303
1,0.0,1.25,-11.360608,-11.126415,16.986205,-10.301332
2,0.0,1.25,-16.238833,-11.722875,9.341086,-16.585621
3,0.0,1.25,-18.929424,-18.026662,14.393226,-13.751677
4,0.0,1.25,-0.096144,-0.096144,23.025851,-2.506192
5,0.0,1.25,-11.360608,-3.508163,17.192054,-10.051577
6,0.0,1.25,-6.589923,-8.278301,15.915499,-12.005828
7,0.0,1.25,-6.493779,-2.728005,19.459270,-7.284907
8,0.0,1.25,-1.552743,-1.551431,21.559208,-4.491303
9,0.0,1.25,-11.360608,-11.126415,16.986205,-10.301332


In [31]:
def likelihood_model(data, error, satisfice, p_prune, prune_t):
    all_likelihoods = np.zeros((len(data.values), 6))

    p_error = error
    softmax_temp = 1
    for i in np.arange(len(data.values)):
        trial = data.values[i]
        click_cost = trial[2]
        all_likelihoods[i, 0] = trial[1] #pid
        all_likelihoods[i, 1] = click_cost
        click_seq = trial[10]
        trial_id = trial[6]
        if click_cost == 0.10:
            rewards = get_rewards_for_trial(rewards_low, trial_id)
        elif click_cost == 1.25:
            rewards = get_rewards_for_trial(rewards_med, trial_id)
        elif click_cost == 4.00:
            rewards = get_rewards_for_trial(rewards_high, trial_id)
        likelihood = strategy_likelihood(click_seq, breadth_first, click_cost, rewards, p_error, softmax_temp, log=True,dist='Uniform', stop=(2, satisfice), pruned=(p_prune, prune_t))
#         strategies = [depth_first, breadth_first, best_first, backwards_best]
#         for j in np.arange(4):
#             strat = strategies[j]
#             likelihood = strategy_likelihood(click_seq, strat, click_cost, rewards, p_error, softmax_temp, log=True, stop=(2, satisfice), pruned=(p_prune, prune_t))
#             all_likelihoods[i, j+2] = likelihood
    return -likelihood

In [113]:
test_data = df[df['info_cost'] == 1.25]
optimize.minimize(lambda x: likelihood_model(test_data, *x), [0.1, 0, 0.5, -10], method='BFGS')

C:\Users\piichan\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
C:\Users\piichan\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in log


      fun: -11.610339115093192
 hess_inv: array([[  3.41331940e+09,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])
      jac: array([  8.34465027e-06,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 150
      nit: 24
     njev: 25
   status: 0
  success: True
        x: array([ -1.20251591e+05,   0.00000000e+00,   5.00000000e-01,
        -1.00000000e+01])

## Misc.

In [ ]:
def strategy_features(state, last_click, action, expansion_rule):
    #returns an array of indicator values for each feature; value is 1 if action is compatible with the strategy
    ''' Features:
        [0] is_term
        [1] term_reward
        [2] voi_myopic
        [3] vpi_action
        [4] vpi_full
        [5] quality_ev
        [6] quality_std
        [7] depth
        [8] same_branch_as_last
    '''
    features = np.zeros(9)
    prob_next_click = expansion_rule(state, [last_click], 'Uniform', None, -30)
    func_name = expansion_rule.__name__
    if func_name == 'best_first':
        max_prob = np.maximum(prob_next_click)
        if prob_next_click[action] == max_prob:
            features[5] = 1
    elif func_name == 'breadth_first':
        if prob_next_click[action] > 0:
            features[7] = -1
    elif func_name == 'depth_first':
        if prob_next_click[action] > 0:
            features[6] = -1
            features[7] = -1
    
    return features

In [ ]:
strategy_features(np.array([0, 1, 2, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]), 2, 3, depth_first)

### Brute Force Approach

In [ ]:
df.sort_values('info_cost')
low_cost = df.loc[df.info_cost == 0.1]
med_cost = df.loc[df.info_cost == 1.25]
high_cost = df.loc[df.info_cost == 4]

In [ ]:
cost_type = low_cost
brfs_count = 0
dfs_count = 0
top = set([9, 10, 11, 12])
left = set([13, 14, 15, 16])
right = set([5,6,7,8])
bottom = set([1, 2, 3, 4])
for seq in cost_type['clicks']:
    if seq == []:
        continue     
    if len(seq) < 4:
        continue
    first_four = seq[:4]
    if set(first_four) == set([1,5,9,13]):
        if len(seq) > 4:
            rest = seq[4:]
            if len(rest) > 4:
                rest = seq[4:8]
            if set(rest) <= set([2,6,10,14]):
                brfs_count += 1 
        else:
            brfs_count += 1
    elif (set(first_four) == top)or(set(first_four) == right)or(set(first_four) == bottom)or(set(first_four) == left):
        if len(seq) > 4:
            rest = set(seq[4:])
            more = False
            if len(rest) > 4:
                more = True
                rest = set(seq[4:8])
            if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                if more:
                    rest = set(seq[8:])
                    if len(rest) > 4:
                        rest = set(seq[8:12])
                    if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                            dfs_count += 1                    
                else:
                    dfs_count += 1
        else:
            dfs_count += 1
    else: #between 1 and 3 clicks were made
        if (set(seq) < top)or(set(seq) < right)or(set(seq)< bottom) or (set(seq) < left):
            dfs_count += 1
        elif set(seq) < set([1, 5, 9, 13]):
            brfs_count += 1
        
(brfs_count/cost_type['clicks'].size) * 100

ver 0.991 

values are given in percentages; top to bottom is low, med, and high cost

i stands for initial (only the first 4 clicks) while f stands for full (entire sequence)

| Breadth-First | Depth-First(i)| Depth-First(f)|
| ------------- |:-------------:| -------------:|
| 3.02          | 12.50         |8.47       |
| 6.07          | 12.13         |    9.375      |
| 4.23          | 3.43          |    2.42       |

